# Analiza przestępczości w Polsce 

In [176]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Ładowanie danych - GUS

In [177]:
df = pd.read_csv('data_crimes.csv',sep=";", quotechar='"')

print(df)

                  Nazwa  1 kwartał;przestępstwa stwierdzone;ogółem;2016;[-]  \
0                POLSKA                                             181162    
1          DOLNOŚLĄSKIE                                              17437    
2    KUJAWSKO-POMORSKIE                                               8804    
3             LUBELSKIE                                               6655    
4              LUBUSKIE                                               6037    
5               ŁÓDZKIE                                              10733    
6           MAŁOPOLSKIE                                              18748    
7           MAZOWIECKIE                                              24765    
8              OPOLSKIE                                               4157    
9          PODKARPACKIE                                               4826    
10            PODLASKIE                                               4114    
11            POMORSKIE                             

### Zmienianie kolumn na wiersze

In [178]:
for i, col in enumerate(df.columns):
    print(f"{i}: {col}")

0: Nazwa
1: 1 kwartał;przestępstwa stwierdzone;ogółem;2016;[-]
2: 1 kwartał;przestępstwa stwierdzone;ogółem;2017;[-]
3: 1 kwartał;przestępstwa stwierdzone;ogółem;2018;[-]
4: 1 kwartał;przestępstwa stwierdzone;ogółem;2019;[-]
5: 1 kwartał;przestępstwa stwierdzone;ogółem;2020;[-]
6: 1 kwartał;przestępstwa stwierdzone;ogółem;2021;[-]
7: 1 kwartał;przestępstwa stwierdzone;ogółem;2022;[-]
8: 1 kwartał;przestępstwa stwierdzone;ogółem;2023;[-]
9: 1 kwartał;przestępstwa stwierdzone;ogółem;2024;[-]
10: 1 kwartał;przestępstwa stwierdzone;o charakterze kryminalnym;2016;[-]
11: 1 kwartał;przestępstwa stwierdzone;o charakterze kryminalnym;2017;[-]
12: 1 kwartał;przestępstwa stwierdzone;o charakterze kryminalnym;2018;[-]
13: 1 kwartał;przestępstwa stwierdzone;o charakterze kryminalnym;2019;[-]
14: 1 kwartał;przestępstwa stwierdzone;o charakterze kryminalnym;2020;[-]
15: 1 kwartał;przestępstwa stwierdzone;o charakterze kryminalnym;2021;[-]
16: 1 kwartał;przestępstwa stwierdzone;o charakterze kryminal

In [179]:
df_long = df.melt(id_vars=['Nazwa'],    #zostawiamy
                  var_name='Okres',     #Nowa Kolumna
                  value_name='Liczba')  #Kolumna dla wartości

df_long

,Nazwa,Okres,Liczba
0,POLSKA,1 kwartał;przestępstwa stwierdzone;ogółem;2016...,181162.0
1,DOLNOŚLĄSKIE,1 kwartał;przestępstwa stwierdzone;ogółem;2016...,17437.0
2,KUJAWSKO-POMORSKIE,1 kwartał;przestępstwa stwierdzone;ogółem;2016...,8804.0
3,LUBELSKIE,1 kwartał;przestępstwa stwierdzone;ogółem;2016...,6655.0
4,LUBUSKIE,1 kwartał;przestępstwa stwierdzone;ogółem;2016...,6037.0
...,...,...,...
10399,ŚLĄSKIE,rok;przestępstwa stwierdzone;z ustawy o przeci...,4876.0
10400,ŚWIĘTOKRZYSKIE,rok;przestępstwa stwierdzone;z ustawy o przeci...,1271.0
10401,WARMIŃSKO-MAZURSKIE,rok;przestępstwa stwierdzone;z ustawy o przeci...,4404.0
10402,WIELKOPOLSKIE,rok;przestępstwa stwierdzone;z ustawy o przeci...,7953.0


In [180]:
column_parts = df_long['Okres'].str.split(';', expand=True)
column_parts

,0,1,2,3,4
0,1 kwartał,przestępstwa stwierdzone,ogółem,2016,[-]
1,1 kwartał,przestępstwa stwierdzone,ogółem,2016,[-]
2,1 kwartał,przestępstwa stwierdzone,ogółem,2016,[-]
3,1 kwartał,przestępstwa stwierdzone,ogółem,2016,[-]
4,1 kwartał,przestępstwa stwierdzone,ogółem,2016,[-]
...,...,...,...,...,...
10399,rok,przestępstwa stwierdzone,z ustawy o przeciwdziałaniu narkomanii,2024,[-]
10400,rok,przestępstwa stwierdzone,z ustawy o przeciwdziałaniu narkomanii,2024,[-]
10401,rok,przestępstwa stwierdzone,z ustawy o przeciwdziałaniu narkomanii,2024,[-]
10402,rok,przestępstwa stwierdzone,z ustawy o przeciwdziałaniu narkomanii,2024,[-]


In [249]:
df_long['Kwartał'] = column_parts[0] 
df_long['Przestępstwo'] = column_parts[2]
df_long['Rok'] = column_parts[3]
df_long['Województwo'] = df_long['Nazwa']

# Final DataFrame
df_final = df_long[['Województwo', 'Przestępstwo', 'Kwartał', 'Rok', 'Liczba']]

# Sorting
finadf_finall_df = df_final.sort_values(['Województwo', 'Kwartał'])

#Saving as csv
df_final.to_csv('transformated_data.csv', index=False, encoding='utf-8')

df_final.head(15)

,Województwo,Przestępstwo,Kwartał,Rok,Liczba
0,POLSKA,ogółem,1 kwartał,2016,181162.0
1,DOLNOŚLĄSKIE,ogółem,1 kwartał,2016,17437.0
2,KUJAWSKO-POMORSKIE,ogółem,1 kwartał,2016,8804.0
3,LUBELSKIE,ogółem,1 kwartał,2016,6655.0
4,LUBUSKIE,ogółem,1 kwartał,2016,6037.0
5,ŁÓDZKIE,ogółem,1 kwartał,2016,10733.0
6,MAŁOPOLSKIE,ogółem,1 kwartał,2016,18748.0
7,MAZOWIECKIE,ogółem,1 kwartał,2016,24765.0
8,OPOLSKIE,ogółem,1 kwartał,2016,4157.0
9,PODKARPACKIE,ogółem,1 kwartał,2016,4826.0


# Preprocessing

In [250]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10404 entries, 0 to 10403
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Województwo   10404 non-null  object 
 1   Przestępstwo  10404 non-null  object 
 2   Kwartał       10404 non-null  object 
 3   Rok           10404 non-null  object 
 4   Liczba        7684 non-null   float64
dtypes: float64(1), object(4)
memory usage: 406.5+ KB


In [251]:
df_final["Rok"] = df_final["Rok"].astype(int)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10404 entries, 0 to 10403
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Województwo   10404 non-null  object 
 1   Przestępstwo  10404 non-null  object 
 2   Kwartał       10404 non-null  object 
 3   Rok           10404 non-null  int64  
 4   Liczba        7684 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 406.5+ KB


C:\Users\Wiola\AppData\Local\Temp\ipykernel_10180\1413867464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Rok"] = df_final["Rok"].astype(int)


### Usuwanie przestępstw łączonych ("razem")

In [252]:
df_final['Przestępstwo'].unique()

array(['ogółem', 'o charakterze kryminalnym',
       'o charakterze gospodarczym',
       'przeciwko bezpieczeństwu powszechnemu i bezpieczeństwu w komunikacji razem',
       'przeciwko bezpieczeństwu powszechnemu i bezpieczeństwu w komunikacji - drogowe',
       'przeciwko życiu i zdrowiu', 'przeciwko mieniu',
       'przeciwko wolności, wolności sumienia, wolności seksualnej i obyczajności razem',
       'przeciwko wolności i wolności sumienia razem',
       'przeciwko wolności', 'przeciwko rodzinie i opiece',
       'przeciwko działalności instytucji państwowych oraz samorządu terytorialnego',
       'przeciwko wymiarowi sprawiedliwości',
       'przeciwko wiarygodności dokumentów',
       'przeciwko obrotowi gospodarczemu',
       'przeciwko obrotowi pieniędzmi i papierami wartościowymi',
       'z ustawy o przeciwdziałaniu narkomanii'], dtype=object)

In [253]:
df_final = df_final[ ~df_final['Przestępstwo'].str.contains("razem", case=False, na=False)]
df_final['Przestępstwo'].unique()

array(['ogółem', 'o charakterze kryminalnym',
       'o charakterze gospodarczym',
       'przeciwko bezpieczeństwu powszechnemu i bezpieczeństwu w komunikacji - drogowe',
       'przeciwko życiu i zdrowiu', 'przeciwko mieniu',
       'przeciwko wolności', 'przeciwko rodzinie i opiece',
       'przeciwko działalności instytucji państwowych oraz samorządu terytorialnego',
       'przeciwko wymiarowi sprawiedliwości',
       'przeciwko wiarygodności dokumentów',
       'przeciwko obrotowi gospodarczemu',
       'przeciwko obrotowi pieniędzmi i papierami wartościowymi',
       'z ustawy o przeciwdziałaniu narkomanii'], dtype=object)

### Missing values

In [254]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8568 entries, 0 to 10403
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Województwo   8568 non-null   object 
 1   Przestępstwo  8568 non-null   object 
 2   Kwartał       8568 non-null   object 
 3   Rok           8568 non-null   int64  
 4   Liczba        6392 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 401.6+ KB


In [255]:
pd.isnull(df_final).sum()

Województwo        0
Przestępstwo       0
Kwartał            0
Rok                0
Liczba          2176
dtype: int64

In [256]:
df_final[df_final.isnull().any(axis=1)]

,Województwo,Przestępstwo,Kwartał,Rok,Liczba
1377,POLSKA,przeciwko wolności,1 kwartał,2016,NaN
1378,DOLNOŚLĄSKIE,przeciwko wolności,1 kwartał,2016,NaN
1379,KUJAWSKO-POMORSKIE,przeciwko wolności,1 kwartał,2016,NaN
1380,LUBELSKIE,przeciwko wolności,1 kwartał,2016,NaN
1381,LUBUSKIE,przeciwko wolności,1 kwartał,2016,NaN
...,...,...,...,...,...
10314,ŚLĄSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2019,NaN
10315,ŚWIĘTOKRZYSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2019,NaN
10316,WARMIŃSKO-MAZURSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2019,NaN
10317,WIELKOPOLSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2019,NaN


#### DROPNA

In [257]:
df_dropna = df_final.copy()
df_dropna.dropna(inplace=True)
pd.isnull(df_dropna).sum()


Województwo     0
Przestępstwo    0
Kwartał         0
Rok             0
Liczba          0
dtype: int64

In [258]:
df_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6392 entries, 0 to 10403
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Województwo   6392 non-null   object 
 1   Przestępstwo  6392 non-null   object 
 2   Kwartał       6392 non-null   object 
 3   Rok           6392 non-null   int64  
 4   Liczba        6392 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 299.6+ KB


### Obliczanie kwartałów

In [323]:
df_final = df_dropna
df_final

,Województwo,Przestępstwo,Kwartał,Rok,Liczba
0,POLSKA,ogółem,1 kwartał,2016,181162.0
1,DOLNOŚLĄSKIE,ogółem,1 kwartał,2016,17437.0
2,KUJAWSKO-POMORSKIE,ogółem,1 kwartał,2016,8804.0
3,LUBELSKIE,ogółem,1 kwartał,2016,6655.0
4,LUBUSKIE,ogółem,1 kwartał,2016,6037.0
...,...,...,...,...,...
10399,ŚLĄSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2024,4876.0
10400,ŚWIĘTOKRZYSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2024,1271.0
10401,WARMIŃSKO-MAZURSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2024,4404.0
10402,WIELKOPOLSKIE,z ustawy o przeciwdziałaniu narkomanii,rok,2024,7953.0


In [324]:
df_final["Kwartał"].unique()

array(['1 kwartał', 'pierwsze półrocze', '1-3 kwartały', 'rok'],
      dtype=object)

In [325]:
#Wyodrębienie okresów czasu, aby obliczyć wszystkie kwartały

Q1 = df_final[df_final["Kwartał"] == "1 kwartał"]
Q12 = df_final[df_final["Kwartał"] == "pierwsze półrocze"]
Q13 =  df_final[df_final["Kwartał"] == "1-3 kwartały"]
Y =  df_final[df_final["Kwartał"] == "rok"]

#Kwartał 2
Q2 = pd.merge(
    Q1, Q12,
    on=["Województwo", "Przestępstwo", "Rok"],
    suffixes=("_Q1", "_Q12")
)

Q2["Liczba"] = Q2["Liczba_Q12"] - Q2["Liczba_Q1"]
Q2["Kwartał"] = "2 kwartał"
Q2= Q2.drop(["Kwartał_Q1", "Liczba_Q1", "Kwartał_Q12", "Liczba_Q12"], axis=1)




#Kwartał 3
Q3 = pd.merge(
    Q12, Q13,
    on=["Województwo", "Przestępstwo", "Rok"],
    suffixes=("_Q12", "_Q13")
)

Q3["Liczba"] = Q3["Liczba_Q13"] - Q3["Liczba_Q12"]
Q3["Kwartał"] = "3 kwartał"
Q3= Q3.drop(["Kwartał_Q12", "Liczba_Q12", "Kwartał_Q13", "Liczba_Q13"], axis=1)


#Kwartał 4
Q4 = pd.merge(
    Q13, Y,
    on=["Województwo", "Przestępstwo", "Rok"],
    suffixes=("_Q13", "_Y")
)

Q4["Liczba"] = Q4["Liczba_Y"] - Q4["Liczba_Q13"]
Q4["Kwartał"] = "4 kwartał"
Q4= Q4.drop(["Kwartał_Q13", "Liczba_Q13", "Kwartał_Y", "Liczba_Y"], axis=1)

df_final = pd.concat([df_final, Q2, Q3, Q4], ignore_index=True)
df_final

,Województwo,Przestępstwo,Kwartał,Rok,Liczba
0,POLSKA,ogółem,1 kwartał,2016,181162.0
1,DOLNOŚLĄSKIE,ogółem,1 kwartał,2016,17437.0
2,KUJAWSKO-POMORSKIE,ogółem,1 kwartał,2016,8804.0
3,LUBELSKIE,ogółem,1 kwartał,2016,6655.0
4,LUBUSKIE,ogółem,1 kwartał,2016,6037.0
...,...,...,...,...,...
11181,ŚLĄSKIE,z ustawy o przeciwdziałaniu narkomanii,4 kwartał,2024,1168.0
11182,ŚWIĘTOKRZYSKIE,z ustawy o przeciwdziałaniu narkomanii,4 kwartał,2024,356.0
11183,WARMIŃSKO-MAZURSKIE,z ustawy o przeciwdziałaniu narkomanii,4 kwartał,2024,429.0
11184,WIELKOPOLSKIE,z ustawy o przeciwdziałaniu narkomanii,4 kwartał,2024,1454.0


In [327]:
df_final["Kwartał"].unique()

array(['1 kwartał', 'pierwsze półrocze', '1-3 kwartały', 'rok',
       '2 kwartał', '3 kwartał', '4 kwartał'], dtype=object)

In [328]:
df_final = df_final[ ~df_final['Kwartał'].str.contains("pierwsze półrocze|1-3 kwartały", case=False, na=False)]
df_final['Kwartał'].unique()

array(['1 kwartał', 'rok', '2 kwartał', '3 kwartał', '4 kwartał'],
      dtype=object)

In [331]:
mapa_kwartał = {
    "1 kwartał": 1,
    "2 kwartał": 2,
    "3 kwartał": 3,
    "4 kwartał": 4,
    "rok": 0
}

df_final["Kwartał"] = df_final["Kwartał"].map(mapa_kwartał)

C:\Users\Wiola\AppData\Local\Temp\ipykernel_10180\1067151899.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Kwartał"] = df_final["Kwartał"].map(mapa_kwartał)


In [333]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7990 entries, 0 to 11185
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Województwo   7990 non-null   object 
 1   Przestępstwo  7990 non-null   object 
 2   Kwartał       7990 non-null   int64  
 3   Rok           7990 non-null   int64  
 4   Liczba        7990 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 374.5+ KB


### Label Encoding

In [334]:
le = LabelEncoder()
df_final['Przestępstwo_ID'] = le.fit_transform(df_final['Przestępstwo'])
df_final['Województwo_ID'] = le.fit_transform(df_final['Województwo'])

C:\Users\Wiola\AppData\Local\Temp\ipykernel_10180\211601003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Przestępstwo_ID'] = le.fit_transform(df_final['Przestępstwo'])
C:\Users\Wiola\AppData\Local\Temp\ipykernel_10180\211601003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Województwo_ID'] = le.fit_transform(df_final['Województwo'])


In [335]:
df_final

,Województwo,Przestępstwo,Kwartał,Rok,Liczba,Przestępstwo_ID,Województwo_ID
0,POLSKA,ogółem,1,2016,181162.0,2,9
1,DOLNOŚLĄSKIE,ogółem,1,2016,17437.0,2,0
2,KUJAWSKO-POMORSKIE,ogółem,1,2016,8804.0,2,1
3,LUBELSKIE,ogółem,1,2016,6655.0,2,2
4,LUBUSKIE,ogółem,1,2016,6037.0,2,3
...,...,...,...,...,...,...,...
11181,ŚLĄSKIE,z ustawy o przeciwdziałaniu narkomanii,4,2024,1168.0,13,15
11182,ŚWIĘTOKRZYSKIE,z ustawy o przeciwdziałaniu narkomanii,4,2024,356.0,13,16
11183,WARMIŃSKO-MAZURSKIE,z ustawy o przeciwdziałaniu narkomanii,4,2024,429.0,13,11
11184,WIELKOPOLSKIE,z ustawy o przeciwdziałaniu narkomanii,4,2024,1454.0,13,12


In [336]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7990 entries, 0 to 11185
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Województwo      7990 non-null   object 
 1   Przestępstwo     7990 non-null   object 
 2   Kwartał          7990 non-null   int64  
 3   Rok              7990 non-null   int64  
 4   Liczba           7990 non-null   float64
 5   Przestępstwo_ID  7990 non-null   int64  
 6   Województwo_ID   7990 non-null   int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 499.4+ KB


# Dane dla Polski

In [337]:
df_poland = df_final[df_final["Województwo"] == "POLSKA"]
df_poland

,Województwo,Przestępstwo,Kwartał,Rok,Liczba,Przestępstwo_ID,Województwo_ID
0,POLSKA,ogółem,1,2016,181162.0,2,9
17,POLSKA,ogółem,1,2017,206988.0,2,9
34,POLSKA,ogółem,1,2018,204889.0,2,9
51,POLSKA,ogółem,1,2019,204517.0,2,9
68,POLSKA,ogółem,1,2020,215963.0,2,9
...,...,...,...,...,...,...,...
11101,POLSKA,z ustawy o przeciwdziałaniu narkomanii,4,2020,14576.0,13,9
11118,POLSKA,z ustawy o przeciwdziałaniu narkomanii,4,2021,14114.0,13,9
11135,POLSKA,z ustawy o przeciwdziałaniu narkomanii,4,2022,14126.0,13,9
11152,POLSKA,z ustawy o przeciwdziałaniu narkomanii,4,2023,13414.0,13,9


### Dane dla Polski przez lata

In [345]:
df_poland_year = df_poland[df_poland["Kwartał"]==0]
df_poland_year

,Województwo,Przestępstwo,Kwartał,Rok,Liczba,Przestępstwo_ID,Województwo_ID
4794,POLSKA,ogółem,0,2016,748459.0,2,9
4811,POLSKA,ogółem,0,2017,753963.0,2,9
4828,POLSKA,ogółem,0,2018,768049.0,2,9
4845,POLSKA,ogółem,0,2019,796557.0,2,9
4862,POLSKA,ogółem,0,2020,765408.0,2,9
...,...,...,...,...,...,...,...
6307,POLSKA,z ustawy o przeciwdziałaniu narkomanii,0,2020,59442.0,13,9
6324,POLSKA,z ustawy o przeciwdziałaniu narkomanii,0,2021,62204.0,13,9
6341,POLSKA,z ustawy o przeciwdziałaniu narkomanii,0,2022,64013.0,13,9
6358,POLSKA,z ustawy o przeciwdziałaniu narkomanii,0,2023,61713.0,13,9


#### Przestępstwa w Polsce przez lata (ogółem)

In [346]:
df_poland_all = df_poland_year[df_poland_year['Przestępstwo']=="ogółem"]
df_poland_all

,Województwo,Przestępstwo,Kwartał,Rok,Liczba,Przestępstwo_ID,Województwo_ID
4794,POLSKA,ogółem,0,2016,748459.0,2,9
4811,POLSKA,ogółem,0,2017,753963.0,2,9
4828,POLSKA,ogółem,0,2018,768049.0,2,9
4845,POLSKA,ogółem,0,2019,796557.0,2,9
4862,POLSKA,ogółem,0,2020,765408.0,2,9
4879,POLSKA,ogółem,0,2021,820846.0,2,9
4896,POLSKA,ogółem,0,2022,862432.0,2,9
4913,POLSKA,ogółem,0,2023,792014.0,2,9
4930,POLSKA,ogółem,0,2024,772681.0,2,9



1. jakie przestęptwa najczściej w polsce jakie najrzadziej - wykres dla crime + który rok + który kwartał
2. wykres dla polski przez lata - ogółem
3. 2022 - skok, czego najwięcej i w jakim województwie - procentowo

4. województwa wszystkie
5. co góruje w każdym województwie a co najrzadziej
6. topki województw ogólnie
7. wskaźniki na województwa

# Dane dla Województw